# This program calculate the difference of joint distribution between America and Japan.
- Use `categorized data`. 
- `america expert` `america personal` `japanese expert` `japanese personal` in `categorized data`.
- results are stored in `results/categorized`.


In [164]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
import os
sns.set(rc={'figure.figsize':(10,9)})

### functions to do all the calculations

In [165]:
def FindBiggestAnswer(nums: list[[int]]):
    # biggest = -1
    # for row in nums:
    #     if row[0] > biggest:
    #         biggest = row[0]
    # return biggest
    return 5

def CalcRowSums(nums: list[[int]]):
    for row in nums:
        sum = 0
        for i in row:
            sum += i
        row.append(sum)

def CalcColSums(nums: list[[int]]):
    nums.append([])
    for col in range(len(nums[0])):
        sum = 0
        for row in range(len(nums) - 1):
            sum += nums[row][col]
        nums[-1].append(sum)

def CalcDelegabilityDistribution(nums: list[[int]]):
    # print(nums)
    new_2darr = copy.deepcopy(nums)
    for row in range(len(new_2darr)-1):
        for col in range(len(new_2darr[0])-1):
            # print(new_2darr[row][col], type(new_2darr[row][col]), new_2darr[row][-1], type(new_2darr[row][-1]))
            if new_2darr[row][-1] != 0:
                new_2darr[row][col] = new_2darr[row][col] / new_2darr[row][-1]
            # print(new_2darr[row][col])
            else:
                new_2darr[row][col] = 0
    return new_2darr

def CalcAnswerDistribution(nums: list[[int]]):
    new_2darr = copy.deepcopy(nums)
    for col in range(len(new_2darr[0])-1):
        for row in range(len(new_2darr)-1):
            if new_2darr[-1][col] != 0:
                new_2darr[row][col] = new_2darr[row][col] / new_2darr[-1][col]
            else:
                new_2darr[row][col] = 0
    return new_2darr

def CalcJointDistribution(nums: list[[int]]):
    new_2darr = copy.deepcopy(nums)
    for row in range(len(new_2darr)-1):
        for col in range(len(new_2darr[0])-1):
            if new_2darr[-1][-1] != 0:
                new_2darr[row][col] = new_2darr[row][col] / new_2darr[-1][-1]
            else:
                new_2darr[row][col] = 0
    return new_2darr

def GenerateArray(df,attribute, label):
    x = []
    for idx in df.index:
        # print(type(df[attribute][idx]))
        # print(int(df[attribute][idx]))
        # if df[attribute][idx] == 'blank':
        #     continue
        # if not isinstance(df[attribute][idx], int):
        #     continue
        # print(type(df[attribute][idx]), type(df[label][idx]))
        x.append([int(df[attribute][idx]), df[label][idx]])
    x_arr = np.array(x)
    answer_label = [[0]*4 for i in range(FindBiggestAnswer(x))]
    for row in x_arr:
        row[0] -= 1
        row[1] -= 1
    for row in x_arr:
        r = row[0]
        c = row[1]
        answer_label[r][c] += 1
    CalcRowSums(answer_label)
    CalcColSums(answer_label)
    # answer_label_arr = np.array(answer_label)
    # print(answer_label_arr)
    return answer_label

### functions before calculate joint distribution

In [166]:
def OutputResults(file, category):
    df = pd.read_csv(category + '/' + file)
    file_name = file.replace('.csv','') # ex: japan_personal_entertainment
    # print(file_name)
    attributes = GetDFAttributes(df)
    # print(attributes)
    for attribute in attributes:
        # print(attribute, '1')
        sub_df = CreateNewDF(attribute, 'Delegability', df)
        if np.isnan(sub_df[attribute][1]):
            continue
        # print(sub_df)
        array = GenerateArray(sub_df, attribute, 'Delegability')
        delegability_distribution = np.array(CalcDelegabilityDistribution(array))
        answer_distribution = np.array(CalcAnswerDistribution(array))
        joint_distribution = np.array(CalcJointDistribution(array))
        # print(label_distribution)
        # print(answer_distribution)
        # print(joint_distribution)
        # Draw(answer_distribution, attribute, file_name, 'answer_distribution')
        # Draw(delegability_distribution, attribute, file_name, 'delegability_distribution')
        Draw(joint_distribution, attribute, file_name, 'joint_distribution')
        

def GetDFAttributes(df):
    attributes = []
    for idx, column in enumerate(df.columns): # df.columns returns all column title of the df
        # idx, column is for example
        '''
        0 Unnamed: 0
        1 task
        2 gender
        3 age
        4 tech_level
        5 edu_level
        6 d_social_skills
        7 d_creativity
        8 d_effort
        9 d_expertise
        10 d_abilities
        11 r_accountable
        12 r_uncertainty
        13 r_impact
        14 t_machine_ability
        15 t_process
        16 t_values
        17 Delegability
        '''
        attributes.append(column)
    attributes = attributes[4:-1]
    return attributes
    # returns a list consists fromthe 4th element to last second element from above, each element is a string

def CreateNewDF(attribute, label, df):
    newDF = df[[attribute, label]]
    return newDF
    # attribute is one column in original table. label is for example delegability
    # returns a new df that has two column, one column is attribute, another column is label.
    # input df is not used in this function

In [167]:
'''
generateFileList(directory)
directory: directory parameter, is the absolute path which contains all categories(america expert/personal/japanese expert/personal)
and csv files.
what to return: a list, each element of this list is an absolute path of a csv file under the input directory.
'''
def generateFileList(directory):
    listResult = []
    for filename in os.listdir(directory):
        if filename == ".DS_Store":
            continue
        inner_dir = directory + "/" + filename
        listResult.append(inner_dir)
    return listResult

In [168]:
'''
findCorrespondingFileInJapanese(america_file)
what this function do: input is an absolute path of a csv file of america.
the return result is the corresponding path of csv file in japanese.
e.g. america_expert_art corresponds to japanese_expert_art
'''
def findCorrespondingFileInJapanese(america_file):
    japan_file = america_file.replace("america_", "japan_")
    japan_file = japan_file.replace("america", "japanese")
    return japan_file

In [169]:
def Draw(distribution_table, attribute, filename):
    idx = []
    for i in range(1,len(distribution_table)):
        idx.append(str(i))
    df = pd.DataFrame(distribution_table[0:-1, 0:-1], index = idx, columns = ['1','2','3','4'])
    chart = sns.heatmap(df, cmap="RdBu", center = 0, annot = True, linewidth = 0.5)
    chart.set_yticklabels(chart.get_yticklabels(), rotation=0)
    chart.set(xlabel = 'delegability', ylabel = 'answer')
    chart.xaxis.tick_top()
    chart.xaxis.set_label_position('top')
    title = attribute
    chart.set_title(title, fontsize = 20, fontweight = 'bold')
    plt.savefig('results/'+filename + "_" + title+'.jpg' , dpi = 100)
    plt.clf()

In [170]:
'''
calculateAndDraw(america_csv_file, japanese_csv_file)
what this function do: input two absolute path of csv file. One file is an america file(
e.g. /Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/america_expert_civic.csv),
the other is the corresponding japanese csv file(
e.g./Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/japanese expert/japan_expert_civic.csv)
calculate the joint distribution of each file
e.g. d_effort for america and d_effort for japanese
then subtract factor of america to factor of japanese
then draw
'''
def calculateAndDrawExpert(america_csv_file, japanese_csv_file):
    df_america = pd.read_csv(america_csv_file)
    df_japan = pd.read_csv(japanese_csv_file)

    filename_america = america_csv_file.replace("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/", "").replace(".csv", "").replace("america_expert_", "")
    # print(filename_america)
    filename_japan = japanese_csv_file.replace("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/japanese expert/", "").replace(".csv", "")

    attributes_america = GetDFAttributes(df_america)
    # print(attributes_america)
    attributes_japan = GetDFAttributes(df_japan)
    for attribute in attributes_america:
        sub_df_america = CreateNewDF(attribute, 'Delegability', df_america)
        if np.isnan(sub_df_america[attribute][1]):
            continue
        sub_df_japan = CreateNewDF(attribute, 'Delegability', df_japan)
        if np.isnan(sub_df_japan[attribute][1]):
            continue
        array_america = GenerateArray(sub_df_america, attribute, 'Delegability')
        array_japan = GenerateArray(sub_df_japan, attribute, 'Delegability')
        joint_distribution_america = np.array(CalcJointDistribution(array_america))
        joint_distribution_japan = np.array(CalcJointDistribution(array_japan))
        
        # print("america_" + filename_america + attribute)
        # print(joint_distribution_america)
        # print("japan_" + filename_japan + attribute)
        # print(joint_distribution_japan)
        # print("================")
        joint_distribution_difference = np.subtract(joint_distribution_america, joint_distribution_japan)
        # print("difference")
        # print(joint_distribution_difference)
        # print("==============")
        Draw(joint_distribution_difference, attribute, filename_america)

def calculateAndDrawPersonal(america_csv_file, japanese_csv_file):
    df_america = pd.read_csv(america_csv_file)
    df_japan = pd.read_csv(japanese_csv_file)

    filename_america = america_csv_file.replace("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america personal/", "").replace(".csv", "").replace("america_personal_", "")
    filename_japan = japanese_csv_file.replace("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/japanese personal/", "").replace(".csv", "")
    print(filename_america)

    attributes_america = GetDFAttributes(df_america)
    # print(attributes_america)
    attributes_japan = GetDFAttributes(df_japan)
    for attribute in attributes_america:
        sub_df_america = CreateNewDF(attribute, 'Delegability', df_america)
        if np.isnan(sub_df_america[attribute][1]):
            continue
        sub_df_japan = CreateNewDF(attribute, 'Delegability', df_japan)
        if np.isnan(sub_df_japan[attribute][1]):
            continue
        array_america = GenerateArray(sub_df_america, attribute, 'Delegability')
        array_japan = GenerateArray(sub_df_japan, attribute, 'Delegability')
        joint_distribution_america = np.array(CalcJointDistribution(array_america))
        joint_distribution_japan = np.array(CalcJointDistribution(array_japan))
        
        # print("america_" + filename_america + attribute)
        # print(joint_distribution_america)
        # print("japan_" + filename_japan + attribute)
        # print(joint_distribution_japan)
        # print("================")
        joint_distribution_difference = np.subtract(joint_distribution_america, joint_distribution_japan)
        # print("difference")
        # print(joint_distribution_difference)
        # print("==============")
        Draw(joint_distribution_difference, attribute, filename_america)
        

In [174]:
main()

civic
living
social
art
entertainment
creative
business
health


<Figure size 720x648 with 0 Axes>

In [172]:
'''
subtractAmericaToJapanese(america_file_list, japanese_file_list)
parameter: america_flie_list and japanese_file_list are list of csv file name.
what this function do:
take the first element in america_file_list, which is a path of a csv file, and take the corresponding element in japanese_file_list, 
which is also a path of a csv file. 
'''
def subtractAmericaToJapaneseExpert(america_file_list, japanese_file_list):
    for america_file in america_file_list:
        america_csv_file = america_file
        japanese_csv_file = findCorrespondingFileInJapanese(america_file)
        calculateAndDrawExpert(america_csv_file, japanese_csv_file)

def subtractAmericaToJapanesePersonal(america_file_list, japanese_file_list):
    for america_file in america_file_list:
        america_csv_file = america_file
        japanese_csv_file = findCorrespondingFileInJapanese(america_file)
        calculateAndDrawPersonal(america_csv_file, japanese_csv_file)
        
        
    

# Main function

## Calculate `america expert` and `japanese expert`, `america personal` and `japanese personal`.

In [173]:
america_expert_directory = r'/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert'
japanese_expert_directory = r'/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/japanese expert'
america_personal_directory = r'/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america personal'
japanese_personal_directory = r'/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/japanese personal'


def main():
    america_expert_file_list = generateFileList(america_expert_directory) # each element in this list is the absolute path of each file in `america_expert_directory`
    america_personal_file_list = generateFileList(america_personal_directory) # each element in this list is the absolute path of each file in `america_personal_directory`
    japanese_expert_file_list = generateFileList(japanese_expert_directory) # each element in this list is the absolute path of each file in `japanese_expert_directory`
    japanese_personal_file_list = generateFileList(japanese_personal_directory) # each element in this list is the absolute path of each file in `japanese_personal_directory`

    # subtractAmericaToJapaneseExpert(america_expert_file_list, japanese_expert_file_list)
    subtractAmericaToJapanesePersonal(america_personal_file_list, japanese_personal_file_list)
    
    # test findCorrespondingFileInJapanese function
    # for america_file in america_expert_file_list:
    #     print(america_file)
    #     pd.read_csv(america_file)
    #     print(findCorrespondingFileInJapanese(america_file))
    #     pd.read_csv(findCorrespondingFileInJapanese(america_file))


In [80]:
# Test function, create dummy file to test
america_expert_directory = r'/Users/gujiaxian/Desktop/论文/project/joint distribution difference/test data/america expert'
japanese_expert_directory = r'/Users/gujiaxian/Desktop/论文/project/joint distribution difference/test data/japanese expert'

def main():
    america_expert_file_list = generateFileList(america_expert_directory)
    print(america_expert_file_list)
    japanese_expert_file_list = generateFileList(japanese_expert_directory)
    subtractAmericaToJapanese(america_expert_file_list, japanese_expert_file_list)

In [151]:
main()

america_/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/america_expert_civictech_level
[[0.00000000e+00 0.00000000e+00 1.72413793e-02 1.72413793e-02
  2.00000000e+00]
 [0.00000000e+00 1.72413793e-02 1.72413793e-02 0.00000000e+00
  2.00000000e+00]
 [3.44827586e-02 3.44827586e-02 2.24137931e-01 0.00000000e+00
  1.70000000e+01]
 [6.89655172e-02 1.03448276e-01 2.75862069e-01 3.44827586e-02
  2.80000000e+01]
 [0.00000000e+00 5.17241379e-02 8.62068966e-02 1.72413793e-02
  9.00000000e+00]
 [6.00000000e+00 1.20000000e+01 3.60000000e+01 4.00000000e+00
  5.80000000e+01]]
japan_/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/japanese expert/japan_expert_civictech_level
[[0.00000000e+00 1.20481928e-02 0.00000000e+00 0.00000000e+00
  1.00000000e+00]
 [0.00000000e+00 1.20481928e-02 3.61445783e-02 1.20481928e-02
  5.00000000e+00]
 [1.20481928e-02 2.04819277e-01 2.65060241e-01 0.00000000e+00
  4.00000000e+01]
 [1.2048

In [49]:
# test directory strip
a = '/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/america_expert_civic.csv'
print(a.strip("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/"))

_expert_civic.csv


In [44]:
# test replace()
a = '/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/america_expert_civic.csv'
a.replace("america_", "japan_")
a.replace("america", "japanese")
b = "jiaxian"
b.replace("gu", "jiajia")
print(b)

# test read_csv
df_america = pd.read_csv("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/america_expert_civic.csv")
print(df_america)

jiaxian
    Unnamed: 0                                               task  gender  \
0            0  Analyzing and controlling the flow of traffic ...  female   
1            1  Analyzing and controlling the flow of traffic ...    male   
2            2  Analyzing and controlling the flow of traffic ...    male   
3            3  Analyzing and controlling the flow of traffic ...  female   
4            4  Analyzing and controlling the flow of traffic ...  female   
5            5  Deciding military actions such as whether to l...    male   
6            6  Deciding military actions such as whether to l...  female   
7            7  Deciding military actions such as whether to l...    male   
8            8  Deciding military actions such as whether to l...    male   
9            9  Deciding military actions such as whether to l...    male   
10          10   Finding and rescuing survivors after earthquakes    male   
11          11   Finding and rescuing survivors after earthquakes  f

In [56]:
# test

file_name = "/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/america_expert_civic.csv"
file_name.replace("/Users/gujiaxian/Desktop/论文/project/joint distribution difference/categorized data/america expert/", "").replace(".csv", "").replace("america_", "")

'expert_civic'